In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import Model, GRB, quicksum

In [2]:
import numpy as np

In [3]:
# Load data
tasks = pd.read_csv("./open_v2/task.csv")
# tasks = tasks[:30]  # for small instance
agvs = pd.read_csv("./open_v2/agv.csv")

In [4]:
agvs

,agv_id,speed_cells_per_sec,max_distance,capacity
0,A001,2,878,7
1,A002,1,875,3
2,A003,2,864,7
3,A004,2,787,5
4,A005,2,867,5
5,A006,1,837,4
6,A007,1,767,7
7,A008,1,452,3
8,A009,1,564,3
9,A010,1,568,7


In [5]:
# depot 좌표
depot = (0, 0)

# 원래 task들을 1 ~ N까지 인덱스로 매핑
points = [(idx+1, row['x'], row['y']) for idx, row in tasks.iterrows()]

# depot을 맨 앞(0)과 마지막(N+1)에 추가
points = [(0, *depot)] + points + [(len(points)+1, *depot)]

# distance[i][j] 딕셔너리 생성
distance = {}
for i_idx, i_x, i_y in points:
    distance[i_idx] = {}
    for j_idx, j_x, j_y in points:
        distance[i_idx][j_idx] = abs(i_x - j_x) + abs(i_y - j_y)


In [6]:
NODE = len(tasks) + 2 # customer nodes + depot(start/end)
K = [k for k in range(0,len(agvs))]
N = [node for node in range(1,len(points)-1)]
N_s = [node for node in range(0,len(points)-1)]
N_t = [node for node in range(0,len(points))]
N_s_t = [node for node in range(0,len(points))]

T_max  =  [tasks['demand'].sum() // agvs.iloc[i,:].capacity for i in range(len(agvs))]

v = agvs['speed_cells_per_sec']
Q = agvs['capacity']
D = agvs['max_distance']
s = list([0])
s +=(list( tasks['service_time']))
s+=list([0])
u = list([0])
u+=(list(tasks['demand']))
u+=list([0])
M = 1000

In [7]:
# # Model
# model = gp.Model("AGV_Routing")

# # 변수 생성
# x = {}
# for k in K:
#     for r in range(T_max[k]):
#         for i in N_s_t:
#             for j in N_s_t:
#                 x[i, j, r] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{j}_{r}")

# y = {}
# for k in K:
#     for r in range(T_max[k]):
#         for i in N:
#             y[i,r] = model.addVar(vtype=GRB.BINARY, name=f"y_{i}_{r}")

# o = {}
# for k in K:
#     for r in range(T_max[k]):
#         for i in N_s_t:
#             o[i,r] = model.addVar(vtype=GRB.INTEGER, name=f"o_{i}_{r}")

# t = {}
# for k in K:
#     for r in range(T_max[k]):
#         for i in N_s_t:
#             t[i,r] = model.addVar(vtype=GRB.CONTINUOUS, name=f"t_{i}_{r}")            

In [8]:
# # 0
# for k in K:
#     for r in range(T_max[k]):
#         model.addConstr(quicksum(x[0,j,r] for j in N ) <= 1)

# # 1 
# for k in K:
#     for r in range(T_max[k]):
#         for j in N:
#             model.addConstr(quicksum(x[i,j,r] for i in N_s if i != j) == quicksum(x[j,i,r] for i in N_t if i != j))
#             model.addConstr(quicksum(x[i,j,r] for i in N_s if i != j) == y[j,r])

# # 2 
# for k in K:
#     for r in range(T_max[k]):
#         for i in N:
#             for j in N:
#                 if i!=j:
#                     model.addConstr(t[j,r]>= t[i,r] + distance[i][j]/v[k] + s[i] - M*(1-x[i,j,r]))

# # 3
# for k in K:
#     for r in range(T_max[k]):
#         for i in N:
#             model.addConstr(u[i] <= o[i,r])

# # # 4
# # for k in K:
# #     for r in range(T_max[k]):
# #         model.addConstr(quicksum(o[i,r] for i in N) <= Q[k])

# for j in N:
#     model.addConstr(
#         quicksum(y[j, r] for k in K for r in range(T_max[k])) >= 1
#     )

# # 4
# for k in K:
#     for r in range(T_max[k]):
#         model.addConstr(quicksum(distance[i][j]*x[i,j,r] for i in N for j in N if i!=j) <= D[k])

In [9]:
# print(1)
# # model.setObjective()


# model.optimize()

In [ ]:
# === MODEL ===
model = gp.Model("AGV_Routing_Small")

x = {}
y = {}
o = {}
t = {}

# === VARIABLES ===
for k in K:
    for r in range(T_max[k]):
        for i in N_s_t:
            for j in N_s_t:
                if i != j:
                    x[k, i, j, r] = model.addVar(vtype=GRB.BINARY, name=f"x_{k}_{i}_{j}_{r}")
        for i in N_s_t:
            y[k, i, r] = model.addVar(vtype=GRB.BINARY, name=f"y_{k}_{i}_{r}")
            o[k, i, r] = model.addVar(vtype=GRB.INTEGER, name=f"o_{k}_{i}_{r}")
            t[k, i, r] = model.addVar(vtype=GRB.CONTINUOUS, name=f"t_{k}_{i}_{r}")

model.update()

# === CONSTRAINTS ===

# 0. Start at most once from depot
for k in K:
    for r in range(T_max[k]):
        model.addConstr(quicksum(x[k, 0, j, r] for j in N if (k, 0, j, r) in x) <= 1)

# 1. Flow conservation & visit indication
for k in K:
    for r in range(T_max[k]):
        for j in N:
            model.addConstr(
                quicksum(x[k, i, j, r] for i in N_s if i != j and (k, i, j, r) in x) ==
                quicksum(x[k, j, i, r] for i in N_t if i != j and (k, j, i, r) in x)
            )
            model.addConstr(
                quicksum(x[k, i, j, r] for i in N_s if i != j and (k, i, j, r) in x) ==
                y[k, j, r]
            )

# 2. Time sequencing with big-M
for k in K:
    for r in range(T_max[k]):
        for i in N:
            for j in N:
                if i != j and (k, i, j, r) in x:
                    model.addConstr(
                        t[k, j, r] >= t[k, i, r] + distance[i][j] / v[k] + s[i] - M * (1 - x[k, i, j, r])
                    )

# 3. Order demand only if visited
for k in K:
    for r in range(T_max[k]):
        for i in N:
            model.addConstr(o[k, i, r] >= u[i] * y[k, i, r])

# 4. Load constraint
for k in K:
    for r in range(T_max[k]):
        model.addConstr(quicksum(o[k, i, r] for i in N) <= Q[k])

# 5. Distance constraint
for k in K:
    for r in range(T_max[k]):
        model.addConstr(
            quicksum(distance[i][j] * x[k, i, j, r]
                     for i in N for j in N if i != j and (k, i, j, r) in x) <= D[k]
        )

# 6. Each task must be served at least once
for j in N:
    model.addConstr(
        quicksum(y[k, j, r] for k in K for r in range(T_max[k])) >= 1
    )

# === OBJECTIVE ===
model.setObjective(
    quicksum(distance[i][j] * x[k, i, j, r]
             for k in K
             for r in range(T_max[k])
             for i in N for j in N if i != j and (k, i, j, r) in x ) + 10,
    GRB.MINIMIZE
)

# === SOLVE ===
model.optimize()


Set parameter Username
Set parameter LicenseID to value 2636023
Academic license - for non-commercial use only - expires 2026-03-13
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-14700, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 28 logical processors, using up to 28 threads

Optimize a model with 6662659 rows, 6927024 columns and 45905900 nonzeros
Model fingerprint: 0xeff99bd8
Variable types: 66606 continuous, 6860418 integer (6793812 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 1.97 seconds (1.86 work units)
Thread count was 1 (of 28 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, g

In [11]:
# === AGV 별 전체 경로 문자열로 저장 ===
from collections import defaultdict

# 경로 재구성
agv_routes = defaultdict(lambda: defaultdict(list))  # agv_routes[k][r] = [(i, j)]

if model.status in [GRB.OPTIMAL, GRB.TIME_LIMIT, GRB.INTERRUPTED]:
    for k in K:
        for r in range(T_max[k]):
            arcs = []
            for i in N_s_t:
                for j in N_s_t:
                    if i != j and (k, i, j, r) in x and x[k, i, j, r].X > 0.5:
                        arcs.append((i, j))
            if arcs:
                agv_routes[k][r] = arcs

    # 각 AGV별 경로 정렬 및 문자열화
    final_paths = []
    for k in agv_routes:
        full_path = []
        used = set()
        for r in sorted(agv_routes[k].keys()):
            arcs = agv_routes[k][r]
            # 경로를 순서대로 정렬
            route = []
            current = 0  # depot에서 시작
            while True:
                next_nodes = [j for (i, j) in arcs if i == current and (i, j) not in used]
                if not next_nodes:
                    break
                next_node = next_nodes[0]
                route.append(next_node)
                used.add((current, next_node))
                current = next_node
            # 노드 이름 포맷 변경: 0 → DEPOT, others → T00XX
            formatted = ["DEPOT"]
            for node in route:
                if node == 0 or node == len(N_s_t) - 1:
                    formatted.append("DEPOT")
                else:
                    formatted.append(f"T{node:04d}")
            full_path.extend(formatted)

        # 중복 DEPOT 제거
        clean_path = [full_path[0]]
        for node in full_path[1:]:
            if node != "DEPOT" or clean_path[-1] != "DEPOT":
                clean_path.append(node)

        final_paths.append({
            "agv_id": f"A{k+1:03d}",
            "route": ",".join(clean_path)
        })

    # CSV 저장
    path_df = pd.DataFrame(final_paths)
    output_path = "agv_routes_formatted.csv"
    path_df.to_csv(output_path, index=False)
    print(f"📁 AGV 경로가 저장되었습니다: {output_path}")
else:
    print("⚠️ 최적 해를 찾지 못했기 때문에 경로를 저장하지 않습니다.")


📁 AGV 경로가 저장되었습니다: agv_routes_formatted.csv
